In [1]:
import json

import boto3
import pandas

In [18]:
def extract_cost_and_usage_data(filename):
    raw_data = None
    service_data = {"Project": [], "Service":[], "Usage": [], "Cost": []}
    
    with open(filename) as file:
        raw_data = json.loads(file.read())
    
    for data in raw_data["ResultsByTime"][0]["Groups"]:
        service_data["Project"].append(data["Keys"][1].replace("ProjectName$", ""))
        service_data["Service"].append(data["Keys"][0])
        service_data["Usage"].append(float(data["Metrics"]["UsageQuantity"]["Amount"]))
        service_data["Cost"].append(float(data["Metrics"]["UnblendedCost"]["Amount"]))

    return pandas.DataFrame.from_dict(service_data)

In [19]:
sandbox_costs = extract_cost_and_usage_data('aws-costs-sandbox.json')
print(sandbox_costs)

                Project                                 Service         Usage  \
0                                                      AWS Glue  1.530443e+00   
1                                    AWS Key Management Service  1.953270e+04   
2                                                    AWS Lambda  2.688288e+04   
3                   CPT                              AWS Lambda  1.456352e+03   
4                                           AWS Secrets Manager  2.390000e+03   
5                                               Amazon DynamoDB  2.298305e+04   
6                           Amazon EC2 Container Registry (ECR)  3.916133e+00   
7                                                   EC2 - Other  2.971061e+01   
8                        Amazon Elastic Compute Cloud - Compute  1.440803e+03   
9                              Amazon Elastic Container Service  4.077486e+03   
10             DATALAKE        Amazon Elastic Container Service  7.828045e+00   
11                          

In [13]:
dev_costs = pandas.read_csv('aws-costs-dev.csv').fillna(0)
test_costs = pandas.read_csv('aws-costs-test.csv').fillna(0)
prod_costs = pandas.read_csv('aws-costs-prod.csv').fillna(0)

In [21]:
all_costs = dev_costs.copy()
for costs in [test_costs, prod_costs]:
    all_costs = all_costs + costs

for column in ["Service", "Neptune($)", "S3($)", "SNS($)"]:
    all_costs[column] = dev_costs[column]

In [15]:
dev_costs

,Service,EC2-ELB($),Relational Database Service($),Neptune($),CloudWatch($),Route 53($),S3($),API Gateway($),Lambda($),Elastic File System($),SNS($),Total cost ($)
0,Service Total,576.665515,529.341462,5.289315,0.744144,0.6,0.018656,0.016065,0.003475,0.000957,7.082000e-07,1112.679591
1,2020-10-01,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000
2,2020-11-01,53.868079,0.000000,0.000000,0.157159,0.0,0.000000,0.000000,0.000000,0.000000,0.000000e+00,54.025238
3,2020-12-01,129.767725,0.000000,0.000000,0.084492,0.0,0.000000,0.000000,0.000000,0.000000,0.000000e+00,129.852217
4,2021-01-01,134.070603,113.843389,0.000000,0.446733,0.0,0.002933,0.014207,0.003475,0.000179,0.000000e+00,248.381519
5,2021-02-01,120.964971,195.662620,0.000000,0.016708,0.0,0.005956,0.001859,0.000000,0.000336,0.000000e+00,316.652450
6,2021-03-01,137.994136,219.835452,5.289315,0.039053,0.6,0.009767,0.000000,0.000000,0.000442,7.082000e-07,363.768167


In [16]:
test_costs

,Service,Relational Database Service($),EC2-ELB($),Route 53($),CloudWatch($),API Gateway($),Elastic File System($),Lambda($),Total cost ($)
0,Service Total,446.953155,277.418248,0.5,0.027236,0.013604,0.000593,0.000021,724.912858
1,2020-10-01,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2,2020-11-01,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
3,2020-12-01,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
4,2021-01-01,34.684070,22.524232,0.0,0.003882,0.000039,0.000028,0.000021,57.212272
5,2021-02-01,195.667442,120.972307,0.0,0.010939,0.013566,0.000282,0.000000,316.664536
6,2021-03-01,216.601644,133.921708,0.5,0.012415,0.000000,0.000283,0.000000,351.036050


In [17]:
prod_costs

,Service,Relational Database Service($),EC2-ELB($),Route 53($),CloudWatch($),API Gateway($),Elastic File System($),Lambda($),Total cost ($)
0,Service Total,350.133550,216.729553,0.6,0.025486,0.010332,0.000426,0.0,567.499347
1,2020-10-01,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000
2,2020-11-01,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000
3,2020-12-01,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000
4,2021-01-01,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000
5,2021-02-01,133.531258,82.807235,0.1,0.011738,0.010206,0.000154,0.0,216.460591
6,2021-03-01,216.602292,133.922319,0.5,0.013748,0.000126,0.000272,0.0,351.038756


In [22]:
all_costs

,API Gateway($),CloudWatch($),EC2-ELB($),Elastic File System($),Lambda($),Neptune($),Relational Database Service($),Route 53($),S3($),SNS($),Service,Total cost ($)
0,0.040001,0.796867,1070.813316,0.001976,0.003496,5.289315,1326.428167,1.7,0.018656,7.082000e-07,Service Total,2405.091796
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000e+00,2020-10-01,0.000000
2,0.000000,0.157159,53.868079,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000e+00,2020-11-01,54.025238
3,0.000000,0.084492,129.767725,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000e+00,2020-12-01,129.852217
4,0.014245,0.450615,156.594836,0.000206,0.003496,0.000000,148.527459,0.0,0.002933,0.000000e+00,2021-01-01,305.593791
5,0.025631,0.039385,324.744513,0.000772,0.000000,0.000000,524.861320,0.1,0.005956,0.000000e+00,2021-02-01,849.777577
6,0.000126,0.065216,405.838163,0.000997,0.000000,5.289315,653.039388,1.6,0.009767,7.082000e-07,2021-03-01,1065.842973
